In [1]:
import numpy as np

In [89]:
n=782
target_list=[]
sample_id_list=[]
logit_list=[]
logprob_list=[]
for i in range(n):
    target_list.append(np.load('output/target_'+str(i)+'.npy'))
    sample_id_list.append(np.load('output/samples_id_'+str(i)+'.npy'))
    logit_list.append(np.load('output/logit_'+str(i)+'.npy'))
    logprob_=np.load('output/logprob_'+str(i)+'.npy')
    logprob_list.append(logprob_.reshape(50, -1).T)

In [90]:
target=np.concatenate(target_list)
sample_id=np.concatenate(sample_id_list)
logit=np.concatenate(logit_list)
logprob=np.concatenate(logprob_list)

In [91]:
np.mean(np.argmax(logit.mean(axis=1), -1)==target)

0.7872

In [157]:
for i in range(20):
    acc=np.mean(np.argmax(np.log(np.sum(np.exp(logit*0.1*i)*np.exp(np.expand_dims(logprob, -1)*0.1*4), axis=1)), -1)==target)
    #acc=np.mean(np.argmax(np.sum(logit*np.exp(logprob.reshape(-1, 50, 1)*0.1*i), axis=1), -1)==target)
    #acc=np.mean(np.argmax(logit[:,i,:], axis=-1)==target)
    print(i*0.1, acc)

0.0 0.00096
0.1 0.79136
0.2 0.79328
0.30000000000000004 0.79408
0.4 0.79488
0.5 0.79504
0.6000000000000001 0.79488
0.7000000000000001 0.79488
0.8 0.79632
0.9 0.79696
1.0 0.79744
1.1 0.79712
1.2000000000000002 0.79728
1.3 0.7976
1.4000000000000001 0.79808
1.5 0.79712
1.6 0.79696
1.7000000000000002 0.79664
1.8 0.79632
1.9000000000000001 0.79616


In [23]:
np.exp(np.expand_dims(logprob.reshape(), -1)*0.5).shape

(40000, 1)

In [79]:
np.exp(logprob)[0]

array([0.00944119, 0.27161035, 0.00825591, 0.01013161, 0.01226162,
       0.01273109, 0.00784586, 0.00803083, 0.00108166, 0.01153791,
       0.01375314, 0.01037615, 0.01026264, 0.01376542, 0.0089599 ,
       0.00115549, 0.00091253, 0.00946964, 0.00080083, 0.01161157,
       0.00114465, 0.01212621, 0.01220938, 0.01148326, 0.00113672,
       0.011853  , 0.00107455, 0.0007487 , 0.01470196, 0.01488104,
       0.01372992, 0.01139705, 0.0088841 , 0.01071398, 0.00912526,
       0.00955774, 0.00933605, 0.01014044, 0.01066163, 0.00108007,
       0.01084374, 0.01095478, 0.00059321, 0.00082864, 0.00104476,
       0.00859163, 0.00113271, 0.0011467 , 0.0114239 , 0.00974295],
      dtype=float32)

In [70]:
logprob[0,1]

-1.3033867

In [49]:
sample_id.shape

(800, 50)

In [50]:
logprob.shape

(40000,)

In [83]:
sample_id.min()

0

In [174]:
def normalize(logprob):
    prob=np.exp(np.array(logprob))
    prob_min=np.min(prob)
    prob_max=np.max(prob)
    return (prob-prob_min)/(prob_max-prob_min)

X=[]
Y=[]

l_n1=[]
c_n1=[]
for j in range(len(sample_id)):
    l=[]
    c=[]
    for i in range(50):
        id_=sample_id[j][i]
        #if 196<=id_<=244:
        if id_<=195:
            l.append(logprob[j][i])
            c.append(float(np.argmax(logit[j][i])==target[j]))
        if id_==245:
            l_n1.append(logprob[j][i])
            c_n1.append(float(np.argmax(logit[j][i])==target[j]))
    std=np.std(l)
    if std>0.5:
        x=normalize(l)
        X.extend(list(x))
        Y.extend(c)
        #Y.append(np.max(c+[c_n1[-1]]))
    
X=np.array(X)
Y=np.array(Y)
l_n1=np.array(l_n1)
c_n1=np.array(c_n1)

In [177]:
c_n1.mean()

0.77376

In [162]:
def cov(X, Y):
    X_normalize=(X-X.mean())/X.std()
    Y_normalize=(Y-Y.mean())/Y.std()
    return np.mean(X_normalize*Y_normalize)
cov(l_n1,c_n1)

0.09604629730726603

In [94]:
14*14

196

In [165]:
np.mean(np.sum(-logprob*np.exp(logprob), axis=-1))

2.160949